### Задание
Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"

Cохранить ее в pandas dataframe

Cохранить pandas dataframe в excel

Cохранить pandas dataframe в pickle

Cохранить pandas dataframe в БД

In [1]:
import requests
import pandas as pd
import time
import openpyxl

In [2]:
objids = []
offset_ = 0
limit_ = 100

while len(objids) < 2000:
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
    try:
        res = requests.get(url)
        objects_data = res.json()

        objects_list = objects_data.get('data').get('list')
        objids.extend([x.get('objId') for x in objects_list])
    except:
        time.sleep(5)
        res = requests.get(url)
        objects_data = res.json()

        objects_list = objects_data.get('data').get('list')
        objids.append([x.get('objId') for x in objects_list])

    offset_ += 100

In [3]:
objids[:10]

[29679, 13383, 13397, 13399, 13398, 13391, 36912, 44076, 23494, 27476]

In [4]:
all_res = []
for uid in objids:
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{uid}'
    try:
        res = requests.get(url)
        if res.ok:
            res = res.json()
            all_res.append(res.get("data"))
    except Exception:
        continue

all_res[0]

{'id': 29679,
 'pdId': 124647,
 'developer': {'devId': 7424,
  'devShortCleanNm': '212-2',
  'devShortNm': '212-2',
  'devFullCleanNm': '212-2',
  'problObjCnt': 0,
  'buildObjCnt': 1,
  'comissObjCnt': 0,
  'regRegionDesc': 'Ростовская область',
  'devPhoneNum': '+7(961)285-90-10',
  'devSite': 'taganrogjsk.wixsite.com',
  'devEmail': 'kirpol76@gmail.com',
  'devInn': '6154572922',
  'devOgrn': '1116154005923',
  'devKpp': '615401001',
  'devLegalAddr': 'Ростовская область, город Таганрог, улица Свободы/Трудовых резервов дом 25/9',
  'devFactAddr': 'обл Ростовская город Таганрог улица Сергея Шило д. 212-2',
  'lastRpdId': 126093,
  'fundGuarantyFlg': 0,
  'devOrgRegRegionCd': 61,
  'devEmplMainFullNm': 'Снимщикова Ольга Викторовна',
  'orgForm': {'id': 20,
   'fullForm': 'Жилищный или жилищно-строительный кооператив',
   'shortForm': 'ЖСК'},
  'objGuarantyEscrowFlg': 0,
  'govFundFlg': 0},
 'region': 61,
 'address': 'г Таганрог',
 'objElemLivingCnt': 18,
 'objReady100PercDt': '2020-03

In [5]:
len(all_res)

2000

In [6]:
houses = pd.DataFrame(data = pd.json_normalize(all_res))

In [7]:
house_data = houses[[
            "region",
            "floorMax",
            "objElemLivingCnt",
            "objReady100PercDt",
            "objSquareLiving",
            "objElemParkingCnt",
            "objPriceAvg"
        ]]

In [8]:
house_data.head()

,region,floorMax,objElemLivingCnt,objReady100PercDt,objSquareLiving,objElemParkingCnt,objPriceAvg
0,61,NaN,18,2020-03-31,1519.00,0,NaN
1,72,20.0,68,2022-12-31,6293.78,31,53109.0
2,72,12.0,65,2024-12-31,6248.51,0,45523.0
3,72,1.0,0,2024-12-31,0.00,125,50768.0
4,72,10.0,43,2024-12-31,4812.98,0,47886.0


In [9]:
writer = pd.ExcelWriter('house_data.xlsx')
house_data.to_excel(writer)
writer.save()

In [10]:
house_data.to_pickle('house_data.pickle')

In [11]:
import sqlite3
con = sqlite3.connect("house_data.db")

In [13]:
house_data.to_sql('house_data_1', con=con)

2000

In [ ]:

for detail_data in all_res[:10]:
    n = 0
    for photo_data in detail_data.get("photoRenderDTO", []):
        photo_url = photo_data.get("objRenderPhotoUrl")

        photo_response = requests.get(photo_url)

        if photo_response.ok:

            with open(f'image_{detail_data.get("id")}_{n}.jpg', 'wb+') as picture:
                picture.write(photo_response.content)
            n += 1